In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
T = 2 * np.pi
n = 1000
h = T / n

Q = [[10, 0, 0],[0, 10, 0],[0, 0, 1]]
R = [[1, 0],[0, 1]]

In [3]:
# Dynamics
def dynamics(x0, u):
    x = np.zeros((n,3))
    x[0] = x0
    for i in range(n-1):
        x[i+1,2] = x[i,2] + h*u[i,1]
        x[i+1,0] = x[i,0] + h*np.cos(x[i,2])*u[i,0]
        x[i+1,1] = x[i,1] + h*np.sin(x[i,2])*u[i,0]
    return x

In [4]:
# Cost Function
def J(x, u):
    intg = 0
    for i in range(len(x)):
        intg += h*(np.dot(np.dot((x[i] - x_g[i]).T, Q), (x[i]-x_g[i])) + np.dot(np.dot(u[i].T, R), u[i]))
    return intg

# Cost Function Directional Derivative
def DJ(x, u, zheta):
    intg = 0
    for i in range(len(x)):
        intg += h*2*(np.dot(np.dot((x[i] - x_g[i]).T, Q), zheta[i, :3]) + np.dot(np.dot(u[i].T, R), zheta[i, 3:]))
    return intg

In [5]:
# Goal Trajectory
x_g = np.zeros((n,3))
for i in range(n):
    x_g[i,0] = 2*h*i/np.pi
    x_g[i,2] = np.pi/2

# Initial Trajectory
x0 = [0,0,np.pi/2]
u_init = np.zeros((n,2))
for i in range(n-1):
    u_init[i] = [1, -0.5]
x_init = dynamics(x0, u_init)

In [6]:
# Initial Cost
J_init = J(x_init, u_init)
print(J_init)

160.37451961348165


In [7]:
# iLQR
eps = 0.1 # Threshold

u_iLQR = u_init
x_iLQR = x_init
zheta = np.zeros((n,5)) # zheta = [z0, z1, z2, v0, v1]

i = 0
while i==0 or np.abs(DJ(x_iLQR, u_iLQR, zheta)) > eps:
    # Get zheta (perturbation)
    # zheta = argmin(DJ(u)*zheta) + 0.5*np.dot(zheta.T, zheta)
    zheta = argminDJ() + # TODO

    # Armijo
    alpha = 0.5
    beta = 0.1
    gamma = 1
    J_u = J(x_iLQR, u_iLQR)
    DJ_u_zheta = DJ(x_iLQR, u_iLQR, zheta)
    while J(dynamics(x0, u_iLQR + gamma*zheta[:, 3:]), u_iLQR + gamma*zheta[:, 3:]) > J_u + alpha*gamma*DJ_u_zheta: # Goal Trajectory may not have same amount of steps as step size varies?
        gamma *= beta

    # Update u, x for all times
    for j in range(len(u_iLQR)):
        u_iLQR[j] += gamma*zheta[j, 3:]
    x_iLQR = dynamics(x0, u_iLQR)
    i += 1


SyntaxError: invalid syntax (1462192039.py, line 12)

In [ ]:
# Plots: Initial Trajectory, Final Optimized Trajectory, Optimized Control Signal
time_discr = np.linspace(0, T, n)

#Plot Initial Trajectory
fig, (ax1, ax2) = plt.subplots(2, 1, height_ratios=[2, 1], figsize=(6, 6))
ax1.plot(x_init[0], x_init[1], label='Initial pos')
ax1.plot(x_g[0], x_g[1], color='red', label='Goal pos')
ax2.plot(time_discr, x_init[2]*180/np.pi, label="\u03B8 Initial")
ax2.plot(time_discr, x_g[2], color='red', label="\u03B8 $_{d}$")
ax1.set_xlabel("x")
ax1.set_ylabel("y")
ax1.set_ylim([-2, 2.1])
ax2.set_xlabel("Time")
ax2.set_ylabel("\u03B8(t) [°]")
ax1.legend()
ax2.legend()
fig.suptitle("Initial Trajectory")
#plt.savefig('ME455_ActiveLearning/HW2/HW2_1_init_traj.png')

#Plots Final Optimized Trajectory
fig2, (ax1, ax2, ax3) = plt.subplots(3, 1)
ax1.step(time_discr, x_iLQR[0], label="x")
ax2.step(time_discr, x_iLQR[1], label="y")
ax1.plot(time_discr, x_g[0], color='red', label=r"$x_{d}$")
ax2.plot(time_discr, x_g[1], color='red', label=r"$y_{d}$")
ax3.step(time_discr, x_iLQR[2], label="\u03B8")
ax3.plot(time_discr, x_g[2], color='red', label="\u03B8$_{d}$")
ax1.set_xlabel("Time")
ax1.set_ylabel("x(t)")
ax2.set_xlabel("Time")
ax2.set_ylabel("y(t)")
ax3.set_xlabel("Time")
ax3.set_ylabel("\u03B8(t) [°]")
ax1.legend()
ax2.legend()
ax3.legend()
fig2.suptitle("Final Optimized Trajectory")
#plt.savefig('ME455_ActiveLearning/HW2/HW2_1_opt_time.png')


fig3, ax = plt.subplots()
ax.step(x_iLQR[0], x_iLQR[1], label='Optimized')
ax.plot(x_g[0], x_g[1], color='red', label='Goal')
ax.legend()
ax.set_ylim([-2, 2])
ax.set_xlabel("x")
ax.set_ylabel("y")
fig3.suptitle("Final Optimized Trajectory")
#plt.savefig('ME455_ActiveLearning/HW2/HW2_1_opt_xy.png')

# Plot Optimized Control Signal
fig4, ax = plt.subplots()
ax.step(time_discr, u_iLQR[0], label=r"u$_{1}$")
ax.step(time_discr, u_iLQR[1], label=r"$u_{2}$")
ax.legend(loc='upper right')
ax.set_xlabel("Time")
ax.set_ylabel(r"$u(t)$")
fig4.suptitle("Optimized Controls")
#plt.savefig('ME455_ActiveLearning/HW2/HW2_1_opt_u.png')

plt.show()

